Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Build TensorFlow Lite Support libraries with Bazel 

In [ ]:
# Create folders
!mkdir -p '/android/sdk'

# Download and move android SDK tools to specific folders
!wget -q 'https://dl.google.com/android/repository/tools_r25.2.5-linux.zip'

!unzip 'tools_r25.2.5-linux.zip'
!mv '/content/tools' '/android/sdk'
# Copy paste the folder
!cp -r /android/sdk/tools /android/android-sdk-linux

# Download NDK, unzip and move contents
!wget 'https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip'

!unzip 'android-ndk-r19c-linux-x86_64.zip'
!mv /content/android-ndk-r19c /content/ndk
!mv '/content/ndk' '/android'
# Copy paste the folder
!cp -r /android/ndk /android/android-ndk-r19c

# Remove .zip files
!rm 'tools_r25.2.5-linux.zip'
!rm 'android-ndk-r19c-linux-x86_64.zip'

# Make android ndk executable to all users
!chmod -R go=u '/android'

In [ ]:
# Clone TensorFlow repository
!git clone https://github.com/tensorflow/tensorflow.git

In [ ]:
# Set and view environment variable
%env PATH = /usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/android/sdk/tools:/android/sdk/platform-tools:/android/ndk
%env ANDROID_SDK_API_LEVEL=29
%env ANDROID_API_LEVEL=29
%env ANDROID_BUILD_TOOLS_VERSION=29.0.2
%env ANDROID_DEV_HOME=/android
%env ANDROID_NDK_API_LEVEL=21
%env ANDROID_NDK_FILENAME=android-ndk-r19c-linux-x86_64.zip
%env ANDROID_NDK_HOME=/android/ndk
%env ANDROID_NDK_URL=https://dl.google.com/android/repository/android-ndk-r19c-linux-x86_64.zip
%env ANDROID_SDK_FILENAME=tools_r25.2.5-linux.zip
%env ANDROID_SDK_HOME=/android/sdk
#%env ANDROID_HOME=/android/sdk
%env ANDROID_SDK_URL=https://dl.google.com/android/repository/tools_r25.2.5-linux.zip

#!echo $PATH
!export -p

In [ ]:
# Update sdk, tools etc
!android update sdk --no-ui -a \
  --filter tools,platform-tools,android-29,build-tools-29.0.2

## Install BAZEL

In [ ]:
'''
# Download Bazel
BAZEL_VERSION = '3.7.2'

!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

# Make script executable
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

# Execute the script file
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

# Extract bazel info
!bazel
'''

# Download Latest version of Bazelisk
!wget https://github.com/bazelbuild/bazelisk/releases/latest/download/bazelisk-linux-amd64

# Make script executable
!chmod +x bazelisk-linux-amd64

# Adding to the path
!sudo mv bazelisk-linux-amd64 /usr/local/bin/bazel

# Extract bazel info
!bazel

In [ ]:
# Clone TensorFlow Lite Support repository
!git clone https://github.com/tensorflow/tflite-support.git

In [ ]:
# Move tensorflow folder
!mv /content/tensorflow /tensorflow
# Rename it
!mv /tensorflow /tensorflow_src

In [ ]:
# Move inside tensorflow folder and run configure
%cd /tensorflow_src
!configs=('/usr/bin/python3''/usr/lib/python3/dist-packages''N''N''N''N''-march=native -Wno-sign-compare''y''/android/sdk')

# Print some info
!printf '%s\n' "${configs[@]}" | ./configure

In [ ]:
# Move one folder up
%cd ../

# Move into tflite-support folder
%cd /content/tflite-support/

!ls

## Build .aar files

In [ ]:
#@title Select library. { display-mode: "form" }

library = '//tensorflow_lite_support/java:tensorflowlite_support.aar' #@param ["//tensorflow_lite_support/java:tensorflowlite_support.aar", "//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/vision:task-library-vision", "//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/text:task-library-text", "//tensorflow_lite_support/java/src/java/org/tensorflow/lite/task/audio:task-library-audio"]
print('You selected:', library)


In [ ]:
#@title Select platform(s). { display-mode: "form" }

platforms = 'arm64-v8a,armeabi-v7a' #@param ["arm64-v8a,armeabi-v7a","x86", "x86_64", "arm64-v8a", "armeabi-v7a","x86,x86_64,arm64-v8a,armeabi-v7a"]
print('You selected:', platforms)


In [ ]:
# Build TensorFlow Lite Support library

!bazel build -c opt --cxxopt='--std=c++14' \
    --config=monolithic \
    --fat_apk_cpu='{platforms}' \
    --host_crosstool_top=@bazel_tools//tools/cpp:toolchain \
    '{library}'

In [ ]:
# Install locate
!apt-get install locate

# Update db
!updatedb

In [ ]:
# Locate files
!locate bazel-bin
!locate *.aar